In [5]:
import pandas as pd
%cd "C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\data" 

[WinError 2] The system cannot find the file specified: 'C:\\Users\\dlais\\OneDrive - University of Illinois Chicago\\Desktop\\BioNLP\\data'
c:\Users\dlais\Downloads


In [6]:
relations = pd.read_csv("msi-relations.tsv", sep='\t', usecols=['name'])['name'].tolist()

def process_dataframe(filename, columns, relation_index):
    df = pd.read_csv(filename, usecols=columns, sep='\t')
    df['triple'] = df.apply(lambda row: (row[columns[0]], relations[relation_index], row[columns[1]]), axis=1)
    return df

df1 = process_dataframe("1_drug_to_protein.tsv", ['node_1_name', 'node_2_name'], 0)
df2 = process_dataframe("2_indication_to_protein.tsv", ['node_1_name', 'node_2_name'], 1)
df3 = process_dataframe("3_protein_to_protein.tsv", ['node_1_name', 'node_2_name'], 2)
df4 = process_dataframe("4_protein_to_biological_function.tsv", ['node_1_name', 'node_2_name'], 3)
df5 = process_dataframe("5_biological_function_to_biological_function.tsv", ['node_1_name', 'node_2_name'], 4)
df6 = process_dataframe("6_drug_indication_df.tsv", ['drug_name', 'indication_name'], 5)

triples = []
for df in [df1, df2, df3, df4, df5, df6]:
    triples.extend(df['triple'].tolist())

file_path = 'triples.txt'
with open(file_path, 'w') as f:
    for triple in triples:
        f.write(' '.join(map(str, triple)) + '\n')

FileNotFoundError: [Errno 2] No such file or directory: 'msi-relations.tsv'

In [ ]:
from tqdm.auto import tqdm
import itertools
import random

## load MRCONSO.RFF (and some basic preprocessing)

In [ ]:
with open(r"C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\umls-2023AB-metathesaurus-full\2023AB\META\MRCONSO.RRF", encoding="utf-8") as f:
    lines1 = f.readlines()
print(len(lines1))

13609918


### use only English names

In [ ]:
cleaned1 = []
count = 0
for l in tqdm(lines1):
    lst = l.rstrip("\n").split("|")
    cui, lang, term, synonym = lst[0], lst[1], lst[2], lst[14]
    if lang != "ENG": continue # comment this out if you need all languages
    row = cui+"||"+term+"||"+synonym.lower()
    cleaned1.append(row)
print(len(cleaned1))

  0%|          | 0/13609918 [00:00<?, ?it/s]

8603906


### remove duplicates

In [ ]:
print(len(cleaned1))
cleaned1 = list(set(cleaned1)) 
print(len(cleaned1))

8603906
6972198


In [ ]:
6972198

## positive pairs generation

In [ ]:
umls_dict = {} # constrauct cui to list of name dict, again
terms_dict = {'P':'primary', 'S':'synonym/secondary'}
for line in tqdm(cleaned1):
    cui, term, name = line.split("||")
    if cui in umls_dict:
        umls_dict[cui].append([terms_dict[term], name])
    else:
        umls_dict[cui] = [[terms_dict[term], name]]

  0%|          | 0/6972198 [00:00<?, ?it/s]

In [ ]:
import json

# Convert umls_dict to JSON
umls_dict_json = json.dumps(umls_dict)

# Save JSON to a file
file_path = r"C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\umls-2023AB-metathesaurus-full\2023AB\umls_dict.json"
with open(file_path, 'w') as f:
    f.write(umls_dict_json)

### generate!

In [ ]:
def gen_pairs(input_list):
    return list(itertools.combinations(input_list, r=2))

In [ ]:
gen_pairs([1,2,3]) # test

[(1, 2), (1, 3), (2, 3)]

In [ ]:
pos_pairs = []
for k,v in tqdm(umls_dict.items()):
    pairs = gen_pairs(v)
    if len(pairs)>50: # if >50 pairs, then trim to 50 pairs
        pairs = random.sample(pairs, 50)
    for p in pairs:
        line = str(k) + "||" + p[0] + "||" + p[1]
        pos_pairs.append(line)

  0%|          | 0/3185657 [00:00<?, ?it/s]

In [1]:
import json

# Specify the file path of the JSON file
file_path = r"C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\umls-2023AB-metathesaurus-full\2023AB\umls_dict.json"

# Load the JSON file
with open(file_path, 'r') as f:
    umls_dict = json.load(f)

In [7]:
# Get the first 10 keys in the dictionary
first_10_keys = list(umls_dict.keys())[:10]

# Display the values of the first 10 keys
for key in first_10_keys:
    print(f"Key: {key}, Value: {umls_dict[key]}")

Key: C1883130, Value: [['primary', 'rat paranasal sinus squamous cell metaplasia'], ['primary', 'squamous cell metaplasia of the rat paranasal sinus']]
Key: C2493202, Value: [['primary', 'medical and surgical @ heart and great vessels @ bypass @ coronary artery, four or more arteries @ percutaneous endoscopic @ intraluminal device']]
Key: C5641721, Value: [['primary', 'miconia fenestrata'], ['synonym/secondary', 'clidemia fenestrata'], ['synonym/secondary', 'clidemia pustulata']]
Key: C2039216, Value: [['synonym/secondary', 'rash in diaper area with scabs'], ['synonym/secondary', 'skin lesion of diaper area with scabs (symptom)'], ['primary', 'skin lesion of diaper area with scabs']]
Key: C0390568, Value: [['synonym/secondary', 'f8c2pc'], ['primary', '3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,10-heptadecafluorodecylphosphocholine']]
Key: C2273100, Value: [['synonym/secondary', 'alsophila dregei'], ['primary', 'cyathea dregei']]
Key: C1850864, Value: [['synonym/secondary', 'familial congenital 

### save the pairwise positive training file

In [ ]:
with open(r'C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\Datasets\pairs.txt', 'w', encoding='utf-8') as f:
    for line in pos_pairs:
        f.write("%s\n" % line)

Negative samples gen.

In [9]:
with open(r"C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\Datasets\output_relations.txt", 'r') as file:
    positive_pairs = file.readlines()

relationships = ["interacts with", "is related to", "interacts", "has function", "is a", "treats"]

entities = {relation:{} for relation in relationships}

for i, positive_pair in enumerate(positive_pairs):
    positive_pair = positive_pair.strip()

    for relation in relationships:
        if relation in positive_pair:
            entity1, entity2 = positive_pair.split(relation)
            entity1, entity2 = entity1.strip(), entity2.strip()
            if entity1 not in entities[relation]: 
                entities[relation][entity1] = []
            entities[relation][entity1].append(entity2)
            break

In [10]:
import random

negative_samples = {relation: {key: {} for key in data} for relation, data in entities.items()}
sample_count = 64

for relation, data in entities.items():
    for key, values in data.items():
        other_keys = list(data.keys())
        for value in values:
            negative_samples[relation][key][value] = []
            while sample_count > 0:
                random_other_key = random.choice(other_keys)
                if random_other_key != key:
                    for other_value in data[random_other_key]:
                        if other_value not in values:
                            negative_samples[relation][key][value].append(other_value)
                            sample_count -= 1
                            if sample_count == 0:
                                break
            sample_count = 64

In [4]:
import json
with open(r"C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\Datasets\negative_samples_count64.json", 'w') as file:
    json.dump(negative_samples, file)

In [12]:
import json
json_file_path = r"C:\Users\dlais\OneDrive - University of Illinois Chicago\Desktop\BioNLP\Datasets\negative_samples_count38.json"
with open(json_file_path, 'r') as file:
    negative_samples = json.load(file)
    

df = []
relationships = ["interacts with", "is related to", "interacts", "has function", "is a", "treats"]
for relation in relationships:
    for entity1, values in negative_samples[relation].items():
        for entity2, negative_entities in values.items():
            df.append(f"{entity1} {relation} {entity2}")
            for negative_entity in negative_entities:
                df.append(f"{entity1} {relation} {negative_entity}")

In [56]:
from datasets import load_dataset
from datasets import Dataset

df = pd.DataFrame(df[:99])
pattern = [1] + [-1]*38
labels = (pattern * (len(df) // len(pattern) + 1))[:len(df)]
df.rename(columns={0: "text"}, inplace=True)
df['labels'] = labels
dataset = Dataset.from_pandas(df).train_test_split(train_size=66, test_size=33, shuffle=False)